# ex

Our final output is a list of Customers containing their data and their coordinates. It can be represented by list of dicts.  
We will get our Customers from XML exercise and their coordinates from `googleapis` (JSON exercise).

---
1. Fetch customers from **http://www.thomas-bayer.com/sqlrest/CUSTOMER/**. Create list named **customers** containing dicts with Customers `id` and `link`.  
    Output should look like this: 
           [{'id': '2', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/'},
            {'id': '3', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/'},
            ... 
           ]
           
2. For each Customer fetch their details using their links. Create list named **customers_data** containing dicts with Customers `id`, `link`, `firstname`, `lastname`, `city` and `street`.  
    Output should look like this:
        [{'city': 'Lyon',
          'firstname': 'Anne',
          'id': '2',
          'lastname': 'Miller',
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/',
          'street': '20 Upland Pl.'},
         {'city': 'San Francisco',
          'firstname': 'Michael',
          'id': '3',
          'lastname': 'Clancy',
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/',
          'street': '542 Upland Pl.'},
          ...
         ]
         
3. For each Customer fetch their coordinates using **http://maps.googleapis.com/maps/api/geocode/json** and Customers data (`"{street}, {city}"`).  
    Create list named **customers_data_geo** containing dicts with Customers `id`, `link`, `firstname`, `lastname`, `city`, `street`, `lat` and `lng`.  
    If Customers location is not found, then fill `lat` and `lng` with `None`, otherwise use first found location.  
    Output should look like this:
        [{'city': 'Lyon',
          'firstname': 'Anne',
          'id': '2',
          'lastname': 'Miller',
          'lat': None,
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/',
          'lng': None,
          'street': '20 Upland Pl.'},
         {'city': 'San Francisco',
          'firstname': 'Michael',
          'id': '3',
          'lastname': 'Clancy',
          'lat': 37.7302685,
          'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/',
          'lng': -122.4659227,
          'street': '542 Upland Pl.'},
          ...
         ]

In [1]:
import requests
import xml.etree.ElementTree as ET

response = requests.get('http://www.thomas-bayer.com/sqlrest/CUSTOMER/')
customers_xml = ET.fromstring(response.text)

customers = customers_xml.getchildren()

customer_temp = []

for customer in customers:
    d = {'id': customer.text,
        'link': customer.get('{http://www.w3.org/1999/xlink}href')}
    
    customer_temp.append(d)


In [2]:
customers = customer_temp
customers

[{'id': '0', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/0/'},
 {'id': '1', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/1/'},
 {'id': '2', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/'},
 {'id': '3', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/'},
 {'id': '4', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/4/'},
 {'id': '5', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/5/'},
 {'id': '6', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/6/'},
 {'id': '7', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/7/'},
 {'id': '8', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/8/'},
 {'id': '9', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/9/'},
 {'id': '10', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/10/'},
 {'id': '11', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/11/'},
 {'id': '12', 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/12/'},
 {'id': '13', 'link': 'http://www.thomas-baye

In [3]:
customers_data = []

for customer in customers: 
    response = requests.get(customer['link'])
    customer_xml = ET.fromstring(response.text)
    firstname = customer_xml.find('FIRSTNAME').text
    lastname = customer_xml.find('LASTNAME').text
    street = customer_xml.find('STREET').text
    city = customer_xml.find('CITY').text
    d = {'id': customer['id'],
        'link': customer['link'],
        'firstname': firstname,
        'lastname': lastname,
        'street': street,
        'city': city} 
    customers_data.append(d)
    
customers_data

[{'city': 'Dallas',
  'firstname': 'Laura',
  'id': '0',
  'lastname': 'Steel',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/0/',
  'street': '429 Seventh Av.'},
 {'city': 'Olten',
  'firstname': 'Susanne',
  'id': '1',
  'lastname': 'King',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/1/',
  'street': '366 - 20th Ave.'},
 {'city': 'Lyon',
  'firstname': 'Anne',
  'id': '2',
  'lastname': 'Miller',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/',
  'street': '20 Upland Pl.'},
 {'city': 'San Francisco',
  'firstname': 'Michael',
  'id': '3',
  'lastname': 'Clancy',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/',
  'street': '542 Upland Pl.'},
 {'city': 'Dallas',
  'firstname': 'Sylvia',
  'id': '4',
  'lastname': 'Ringer',
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/4/',
  'street': '365 College Av.'},
 {'city': 'Paris',
  'firstname': 'Laura',
  'id': '5',
  'lastname': 'Miller',
  'link': 'http://www.thomas-bayer.com/sqlrest/C

In [5]:
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'

cust_temp = customers_data[0]
street = cust_temp['street'] if cust_temp['street'] is not None else ''  
city = cust_temp['city'] if cust_temp['city'] is not None else ''  
customer_params = {'address': street + ', ' + city}

print(city + ' - ' + street)

response = requests.get(base_url, params = customer_params)

results = response.json()['results']

lat = None  
lng = None  
if len(results) > 0:  
    cust_geo = results[0]['geometry']['location']
    lat = cust_geo['lat']
    lng = cust_geo['lng']

cust_temp['lat'] = lat
cust_temp['lng'] = lng
cust_temp
#customers_data

Dallas - 429 Seventh Av.


{'city': 'Dallas',
 'firstname': 'Laura',
 'id': '0',
 'lastname': 'Steel',
 'lat': 32.7529735,
 'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/0/',
 'lng': -96.817878,
 'street': '429 Seventh Av.'}

In [6]:
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
customers_data_geo = []

for customer in customers_data:
    street = customer['street'] if customer['street'] is not None else ''  
    city = customer['city'] if customer['city'] is not None else ''  
    customer_params = {'address': street + ', ' + city}
    response = requests.get(base_url, params = customer_params)
    results = response.json()['results']
    lat = None  
    lng = None
    if len(results) > 0:  
        cust_geo = results[0]['geometry']['location']
        lat = cust_geo['lat']
        lng = cust_geo['lng']
    
    customer_copy = customer.copy()
    customer_copy['lat'] = lat
    customer_copy['lng'] = lng
    customers_data_geo.append(customer_copy)
    
customers_data_geo
    


[{'city': 'Dallas',
  'firstname': 'Laura',
  'id': '0',
  'lastname': 'Steel',
  'lat': 32.7529735,
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/0/',
  'lng': -96.817878,
  'street': '429 Seventh Av.'},
 {'city': 'Olten',
  'firstname': 'Susanne',
  'id': '1',
  'lastname': 'King',
  'lat': None,
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/1/',
  'lng': None,
  'street': '366 - 20th Ave.'},
 {'city': 'Lyon',
  'firstname': 'Anne',
  'id': '2',
  'lastname': 'Miller',
  'lat': None,
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/2/',
  'lng': None,
  'street': '20 Upland Pl.'},
 {'city': 'San Francisco',
  'firstname': 'Michael',
  'id': '3',
  'lastname': 'Clancy',
  'lat': 37.7302685,
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOMER/3/',
  'lng': -122.4659227,
  'street': '542 Upland Pl.'},
 {'city': 'Dallas',
  'firstname': 'Sylvia',
  'id': '4',
  'lastname': 'Ringer',
  'lat': 35.3111223,
  'link': 'http://www.thomas-bayer.com/sqlrest/CUSTOME